In [1]:
import glob
import pathlib
import re
import shutil
from pathlib import Path

import cv2
import h5py
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from ultralytics import YOLO

In [2]:
bounding_box_df = pd.read_csv("datasets/wacv2024_ictrap_dataset/3_m-h-h_annotation.csv")

In [3]:
cap = cv2.VideoCapture("datasets/wacv2024_ictrap_dataset/3_m-h-h_dvs.mp4")

In [4]:
bounding_box_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15052 entries, 0 to 15051
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   frame_index  15052 non-null  int64  
 1   is_keyframe  15052 non-null  bool   
 2   class        15052 non-null  object 
 3   confidence   15052 non-null  object 
 4   left         15052 non-null  float64
 5   top          15052 non-null  float64
 6   width        15052 non-null  float64
 7   height       15052 non-null  float64
 8   center_x     15052 non-null  float64
 9   center_y     15052 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(2)
memory usage: 1.0+ MB


In [5]:
point_cloud_dir = "datasets/insect_combined/full_trajectories/mu-3/"
point_cloud_dir = pathlib.Path(point_cloud_dir).as_posix()

In [6]:
point_clouds = glob.glob(f"{point_cloud_dir}/*.csv")

In [7]:
# Add the end time to the filename if not already present.
# This makes it easier to skip point clouds that lay outside a frame time.

regex = re.compile(r"\d+")

for point_cloud in point_clouds:
    if not "end" in point_cloud:
        df = pd.read_csv(point_cloud)
        start = int(regex.findall(point_cloud)[-1])
        end = start + df.loc[:, 't'].max()

        file = pathlib.Path(point_cloud)
        file.rename(file.with_name(file.stem + f"_end{end}.csv"))

point_clouds = glob.glob(f"{point_cloud_dir}/*.csv")

In [8]:
# create list with unique frame indices.
frames_with_bbs = bounding_box_df.loc[:, 'frame_index'].unique()
regex = re.compile(r"\d+")
consolidated_dict = []

# Open HDF5 file containing the external triggers and the corresponding frames.
with h5py.File("./datasets/wacv2024_ictrap_dataset/3_m-h-h.h5", "r") as f:
    ext_triggers_negative = f['EXTERNAL_TRIGGERS']['corrected_negative']
    ext_triggers_positive = f['EXTERNAL_TRIGGERS']['corrected_positive']

    for frame_number in frames_with_bbs:
        start_exposure = ext_triggers_negative[ext_triggers_negative['f'] == frame_number]['t']
        end_exposure = ext_triggers_positive[ext_triggers_positive['f'] == frame_number]['t']

        df = bounding_box_df.loc[bounding_box_df['frame_index'] == frame_number]
        df_dict = df.to_dict(orient='records')

        # If no data is available on this frame index, skip it.
        if start_exposure.size == 0 or end_exposure.size == 0:
            df_dict[i]['class'] = "no class"
            consolidated_dict.append(df_dict[i])
            continue

        # Load the start and end exposure time for the current frame.
        start_exposure = start_exposure.item()
        end_exposure = end_exposure.item()
        # create a separate dataframe for the current frame.
        df = bounding_box_df.loc[bounding_box_df['frame_index'] == frame_number]
        df_dict = df.to_dict(orient='records')

        # Set frame position of video
        # cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        # res, frame = cap.read()
        # for _, item in df.iterrows():
        #     frame = cv2.rectangle(frame, (int(item['left']), int(item['top'])), (int(item['left'] + item['width']), int(item['top'] + item['height'])), color=(0, 0, 255))
        # cv2.imshow("image", frame)
        # cv2.waitKey(0)

        # Dictionary to store bounding boxe hits in
        bb_match_dict = {}

        x0 = df['left'].to_numpy()
        y0 = df['top'].to_numpy()
        x1 = x0 + df['width'].to_numpy()
        y1 = y0 + df['height'].to_numpy()

        for point_cloud in point_clouds:
            start = int(regex.findall(point_cloud)[-2])
            end = int(regex.findall(point_cloud)[-1])

            if not (start_exposure > end or end_exposure < start):
                points = pd.read_csv(point_cloud)

                # Only take points that fall inside the exposure time of the selected frame
                points = points[((points['t'] + start) >= start_exposure) & ((points['t'] + start) <= end_exposure)]

                # Extract the x and y coordinate columns
                points = points.loc[:, ['x', 'y']].to_numpy()
                if points.size == 0:
                    continue
                # Create an array for the x and y coordinates in shape of (n,1)
                points_x = points[:, 0][:, None]
                points_y = points[:, 1][:, None]

                matches = pd.DataFrame((points_x >= x0) & (points_x <= x1) & (points_y >= y0) & (points_y <= y1))
                matches = matches.apply(pd.Series.value_counts)
                bb_match_dict[point_cloud] = matches.replace(np.nan, 0)

        for i in range(len(df.index)):
            counter = ["", 0]
            for key, value in bb_match_dict.items():
                if True in value[i] and int(value[i][True]) > counter[1]:
                    counter[0] = key
                    counter[1] = value[i][True]

            # Correct the class to the correct insect
            if not counter[0]:
                df_dict[i]['class'] = "no class"
            else:
                df_dict[i]['class'] = counter[0].split("_")[-4]
            consolidated_dict.append(df_dict[i])



In [9]:
consolidated_df = pd.DataFrame(consolidated_dict)
consolidated_df.to_csv("./datasets/consolidated_data.csv", index=False)

# Creating the dataset

In [24]:
consolidated_df = pd.read_csv("./datasets/consolidated_data.csv")

In [25]:
# Normalize bounding box values
img_width = 1280
img_height = 720
consolidated_df.loc[:, 'center_x'] = consolidated_df.loc[:, 'center_x'] / img_width
consolidated_df.loc[:, 'width'] = consolidated_df.loc[:, 'width'] / img_width
consolidated_df.loc[:, 'center_y'] = consolidated_df.loc[:, 'center_y'] / img_height
consolidated_df.loc[:, 'height'] = consolidated_df.loc[:, 'height'] / img_height

In [26]:
consolidated_df

,frame_index,is_keyframe,class,confidence,left,top,width,height,center_x,center_y
0,0,True,bee,certain,638.40,83.40,0.025312,0.044167,0.511406,0.137917
1,0,True,bee,certain,345.60,151.80,0.012188,0.020000,0.276094,0.220833
2,1,False,bee,certain,635.85,83.25,0.025312,0.044167,0.509414,0.137708
3,1,False,no class,certain,345.80,150.20,0.012344,0.020556,0.276328,0.218889
4,2,False,bee,certain,633.30,83.10,0.025312,0.044167,0.507422,0.137500
...,...,...,...,...,...,...,...,...,...,...
15046,8199,False,bee,uncertain,128.80,372.80,0.010781,0.017500,0.106016,0.526528
15047,8200,True,bee,uncertain,441.60,141.60,0.026250,0.041667,0.358125,0.217500
15048,8200,False,bee,uncertain,127.40,373.60,0.010781,0.017500,0.104922,0.527639
15049,8201,True,no class,uncertain,126.00,374.40,0.010781,0.017500,0.103828,0.528750


In [27]:
consolidated_df = consolidated_df[consolidated_df.loc[:, 'class'] != "no class"]

In [28]:
consolidated_df.loc[:, 'class'].value_counts()

class
bee    11379
ins      810
Name: count, dtype: int64

In [29]:
consolidated_df[consolidated_df['is_keyframe'] == True].loc[:, 'class'].value_counts()

class
bee    3172
ins     530
Name: count, dtype: int64

In [30]:
# Paths
img_path = Path("./datasets/dvs_frames/")
dataset = Path("./datasets/consolidated_data/")

In [1057]:
img_path.mkdir(parents=True, exist_ok=True)

vidcap = cv2.VideoCapture("./datasets/wacv2024_ictrap_dataset/3_m-h-h_dvs.mp4")
success, image = vidcap.read()
count = 0

while success:
    cv2.imwrite(img_path / f"frame_{count}.jpg", image)
    success, image = vidcap.read()
    count += 1

In [21]:
seed = 42

In [389]:
## Use GroupShuffleSplit since there could be multiple rows in the same dataframe with the same frame index. We want to keep these together.
# First split df into train and test-validation (80%, 20%)
splitter = GroupShuffleSplit(test_size=.2, n_splits=10, random_state=seed)
train_ids, test_validation_ids = next(splitter.split(consolidated_df, groups=consolidated_df.loc[:, 'frame_index']))

# Split the remaining half down the middle for the separate test and validation df.
splitter = GroupShuffleSplit(test_size=.5, n_splits=10, random_state=seed)
test_ids, validation_ids = next(splitter.split(consolidated_df.iloc[test_validation_ids],
                                               groups=consolidated_df.iloc[test_validation_ids].loc[:, 'frame_index']))

train = consolidated_df.iloc[train_ids]
validation = consolidated_df.iloc[test_validation_ids].iloc[validation_ids]
test = consolidated_df.iloc[test_validation_ids].iloc[test_ids]

# Although not stratified, the distribution percentages are about the same.
print(train.loc[:, 'class'].value_counts(normalize=True) * 100)
print(validation.loc[:, 'class'].value_counts(normalize=True) * 100)
print(test.loc[:, 'class'].value_counts(normalize=True) * 100)

class
bee    93.327193
ins     6.672807
Name: proportion, dtype: float64
class
bee    93.354943
ins     6.645057
Name: proportion, dtype: float64
class
bee    93.581081
ins     6.418919
Name: proportion, dtype: float64
       frame_index  is_keyframe class confidence        left         top  \
16               8         True   bee    certain  618.000000   82.200000   
17               8         True   bee    certain  346.200000  150.000000   
28              14         True   bee    certain  606.600000   84.000000   
29              14        False   bee    certain  349.800000  148.457143   
38              19        False   bee    certain  607.885714   90.857143   
...            ...          ...   ...        ...         ...         ...   
14944         8125        False   bee  uncertain  400.896774  192.309677   
14971         8152        False   bee  uncertain  401.400000  190.628571   
14990         8171        False   bee  uncertain  405.400000  185.600000   
15031         8192   

In [1186]:
# Create the necessary paths.
(dataset / "images/train").mkdir(parents=True, exist_ok=True)
(dataset / "images/val").mkdir(parents=True, exist_ok=True)
(dataset / "images/test").mkdir(parents=True, exist_ok=True)
(dataset / "labels/train").mkdir(parents=True, exist_ok=True)
(dataset / "labels/val").mkdir(parents=True, exist_ok=True)
(dataset / "labels/test").mkdir(parents=True, exist_ok=True)

In [16]:
frames = glob.glob(str(img_path / "*.jpg"))

In [17]:
def copy_bounding_box_tx(df, path):
    with open(path, "w") as f:
        for _, row in df.iterrows():
            if row.loc['class'] == "ins":
                class_nr = 0
            elif row.loc['class'] == "bee":
                class_nr = 1

            f.write(f"{class_nr} {row.loc['center_x']} {row.loc['center_y']} {row.loc['width']} {row.loc['height']}\n")

In [32]:
for frame in frames:
    index = int(frame.split("\\")[-1].split("_")[-1].split(".")[0])

    if not train[train.loc[:, 'frame_index'] == index].empty:
        train_sub_df = train[train.loc[:, 'frame_index'] == index]

        shutil.copy(frame, dataset / "images/train")
        copy_bounding_box_tx(train_sub_df, dataset / f"labels/train/frame_{index}.txt")

    elif not validation[validation.loc[:, 'frame_index'] == index].empty:
        validation_sub_df = validation[validation.loc[:, 'frame_index'] == index]

        shutil.copy(frame, dataset / "images/val")
        copy_bounding_box_tx(validation_sub_df, dataset / f"labels/val/frame_{index}.txt")

    elif not test[test.loc[:, 'frame_index'] == index].empty:
        test_sub_df = test[test.loc[:, 'frame_index'] == index]

        shutil.copy(frame, dataset / "images/test")
        copy_bounding_box_tx(test_sub_df, dataset / f"labels/test/frame_{index}.txt")

# Training the model

In [6]:
model = YOLO("yolo11n.pt")

In [8]:
results = model.train(data="consolidated_data.yaml", epochs=1000, imgsz=640, project="yolo_live_labeling", name="1000epochs_imgsz640", plots=True, resume=False, save_period=50)

New https://pypi.org/project/ultralytics/8.3.200 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.178  Python-3.12.11 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=consolidated_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1000, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=1000epochs_imgsz640, nbs=64, nms=False, op

train: Scanning C:\Users\Bas_K\source\repos\thesis-insect-monitoring\datasets\consolidated_data\labels\train.cache... 5080 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5080/5080 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 188.5139.4 MB/s, size: 25.3 KB)


val: Scanning C:\Users\Bas_K\source\repos\thesis-insect-monitoring\datasets\consolidated_data\labels\val.cache... 636 images, 0 backgrounds, 0 corrupt: 100%|██████████| 636/636 [00:00<?, ?it/s]


Plotting labels to yolo_live_labeling\1000epochs_imgsz640\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_live_labeling\1000epochs_imgsz640
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

   961/1000      4.21G     0.8491      0.432     0.7952         24        640: 100%|██████████| 318/318 [00:20<00:00, 15.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:01<00:00, 13.22it/s]

                   all        636       1234      0.864      0.691      0.775      0.453
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 861, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



961 epochs completed in 6.205 hours.
Optimizer stripped from yolo_live_labeling\1000epochs_imgsz640\weights\last.pt, 5.6MB
Optimizer stripped from yolo_live_labeling\1000epochs_imgsz640\weights\best.pt, 5.6MB

Validating yolo_live_labeling\1000epochs_imgsz640\weights\best.pt...
Ultralytics 8.3.178  Python-3.12.11 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:01<00:00, 10.42it/s]


                   all        636       1234      0.865      0.702      0.778      0.459
                insect         78         82      0.819      0.512      0.618      0.348
                   bee        620       1152      0.911      0.891      0.938      0.571
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to yolo_live_labeling\1000epochs_imgsz640


In [33]:
model.val(data="consolidated_data.yaml", split="test")

Ultralytics 8.3.178  Python-3.12.11 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 186.8149.6 MB/s, size: 27.8 KB)


val: Scanning C:\Users\Bas_K\source\repos\thesis-insect-monitoring\datasets\consolidated_data\labels\test... 635 images, 0 backgrounds, 0 corrupt: 100%|██████████| 635/635 [00:00<00:00, 4096.42it/s]

val: New cache created: C:\Users\Bas_K\source\repos\thesis-insect-monitoring\datasets\consolidated_data\labels\test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 40/40 [00:02<00:00, 19.05it/s]


                   all        635       1184       0.85      0.671      0.776      0.447
                insect         73         76      0.781      0.487      0.632      0.327
                   bee        619       1108      0.919      0.856       0.92      0.566
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to yolo_live_labeling\1000epochs_imgsz6404


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000207FCF06450>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [390]:
model_insect = YOLO("./yolo_live_labeling/1000epochs_imgsz640/weights/best.pt")

In [394]:
results = model.track("./datasets/wacv2024_ictrap_dataset/2_l-h-l_dvs.mp4", show=True, conf=0.7)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/34863) C:\Users\Bas_K\source\repos\thesis-insect-monitoring\datasets\wacv2024_ictrap_dataset\2_l-h-l_dvs.mp4: 384x640 (no detections), 11.7ms
video 1/1 (frame 2/34863) C:\Users\Bas_K\source\repos\thesis-insect-monitoring\datasets\wacv2024_ictrap_dataset\2_l-h-l_dvs.mp4: 384x640 (no detections), 12.1ms
video 1/1 (frame 3/34863) C:\Users\Bas_K\source\repos\thesis-insect-monitoring\datasets\wacv2024_ictrap_dataset\2_l-h-l_dvs.mp4: 384x640 

KeyboardInterrupt: 

In [395]:
from collections import defaultdict

# Open the video file
video_path = "./datasets/wacv2024_ictrap_dataset/2_l-h-l_dvs.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        result = model_insect.track(frame, persist=True)[0]

        # Get the boxes and track IDs
        if result.boxes and result.boxes.is_track:
            boxes = result.boxes.xywh.cpu()
            track_ids = result.boxes.id.int().cpu().tolist()

            # Visualize the result on the frame
            frame = result.plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 5.7ms
Speed: 1.2ms preprocess, 5.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.2ms
Speed: 2.5ms preprocess, 5.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.9ms
Speed: 1.1ms preprocess, 5.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 1.1ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.0ms
Speed: 1.3ms preprocess, 5.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 1.1ms preprocess, 7.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.1ms
Speed: 1.1ms preprocess, 6.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 2.3ms 

KeyboardInterrupt: 